## Das Module multiprocessing

Erstellen Sie zwei parallelisierte Implementierung ihrer $\pi$-Bistimmung nach Monte Carlo, wovon eine auf *map* und eine auf *Queues* basiert. Vergleichen Sie die Rechenzeiten mit denen ihrer bisherigen Version. Wurden Ihre Erwartungen bezüglich der Beschleunigung erfüllt?

In [1]:
%load_ext cython

In [2]:
%%cython -a
from libc.math cimport hypot
from libc.stdlib cimport rand, srand, RAND_MAX

#@cython.cdivision(True)
cpdef monte_carlo(int quantity, int seed):
    cdef int hits = 0
    cdef double width = 0.0
    cdef double height = 0.0
    cdef double temp = 0.0
    srand(seed)
    
    for _ in range(quantity):
        width = rand()/RAND_MAX
        height = rand()/RAND_MAX
        temp = hypot(width, height)
        hits = hits + int(temp // 1)

    return [hits, quantity]

In [3]:
#Monte Carlo Test
import random

random_number = int(random.random() * (2**31 -1))
print(monte_carlo(2_500_000, random_number))

[536426, 2500000]


In [4]:
def worker_monte_carlo(q_in, q_out):
    while True:
        arguments = q_in.get()
        x, y = arguments
        result = monte_carlo(x,y)
        q_out.put(result)
        q_in.task_done()

In [5]:
import multiprocessing
import math
import random
nrOfCores = multiprocessing.cpu_count()
print('nrOfCores:', nrOfCores)

l = []
for i in range (128):
    random_number = int(random.random() * (2**31 -1))
    l.append((2_500_000, random_number))

nrOfCores: 8


In [6]:
print(l)

[(2500000, 2014582756), (2500000, 1560866857), (2500000, 1066416651), (2500000, 1466258522), (2500000, 1440670760), (2500000, 124494017), (2500000, 1818363175), (2500000, 1959691617), (2500000, 1730830421), (2500000, 1356325954), (2500000, 692810410), (2500000, 223273857), (2500000, 1889299044), (2500000, 808019053), (2500000, 1300235799), (2500000, 1541904986), (2500000, 2145061717), (2500000, 688979701), (2500000, 692618914), (2500000, 312874190), (2500000, 591960289), (2500000, 957250534), (2500000, 936852740), (2500000, 988764501), (2500000, 1568520451), (2500000, 1713404497), (2500000, 720430390), (2500000, 428301317), (2500000, 2000542047), (2500000, 304607639), (2500000, 1758391222), (2500000, 1947099775), (2500000, 1059842299), (2500000, 1481131343), (2500000, 724407065), (2500000, 620119752), (2500000, 2098475763), (2500000, 487953385), (2500000, 232653352), (2500000, 1186984153), (2500000, 661580492), (2500000, 260312167), (2500000, 520385320), (2500000, 1409966484), (2500000

In [11]:
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for result_tuple in result_list:
    tuple_hits, tuple_trys = result_tuple
    hits += tuple_hits
    trys += tuple_trys
    
print((trys-hits)/trys * 4)
for p in processes:
    p.terminate()

[[536375, 2500000], [537557, 2500000], [535980, 2500000], [536084, 2500000], [536483, 2500000], [536580, 2500000], [536146, 2500000], [536749, 2500000], [536536, 2500000], [536430, 2500000], [536540, 2500000], [537546, 2500000], [535835, 2500000], [536903, 2500000], [537304, 2500000], [536315, 2500000], [536996, 2500000], [537279, 2500000], [536814, 2500000], [536304, 2500000], [535578, 2500000], [536734, 2500000], [535432, 2500000], [536146, 2500000], [537326, 2500000], [536413, 2500000], [536498, 2500000], [536261, 2500000], [536578, 2500000], [536081, 2500000], [535637, 2500000], [536911, 2500000], [536802, 2500000], [536081, 2500000], [536642, 2500000], [537007, 2500000], [537168, 2500000], [536715, 2500000], [536030, 2500000], [536979, 2500000], [536128, 2500000], [536712, 2500000], [535415, 2500000], [536383, 2500000], [536016, 2500000], [535367, 2500000], [536337, 2500000], [536079, 2500000], [536291, 2500000], [535809, 2500000], [536745, 2500000], [536967, 2500000], [537523, 25

## TimeIt

In [8]:
%%timeit
in_queue = multiprocessing.JoinableQueue()
result_queue = multiprocessing.Queue()

processes = []
for i in range(nrOfCores):
    p = multiprocessing.Process(target = worker_monte_carlo, 
                                args = (in_queue, result_queue))
    processes.append(p)
    p.start()
    
for parameter_set in l:
    in_queue.put(parameter_set)
    
import time

in_queue.join()

result_list = []
while not result_queue.empty():
    result_list.append(result_queue.get())

print(result_list)

hits = 0
trys = 0

for result_tuple in result_list:
    tuple_hits, tuple_trys = result_tuple
    hits += tuple_hits
    trys += tuple_trys
    
print((trys-hits)/trys * 4)
for p in processes:
    p.terminate()
    
#5.96 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#6.03 s ± 70.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.9 s ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

[[536483, 2500000], [536540, 2500000], [536814, 2500000], [536430, 2500000], [536146, 2500000], [535980, 2500000], [536146, 2500000], [536375, 2500000], [537546, 2500000], [537557, 2500000], [536315, 2500000], [536749, 2500000], [536580, 2500000], [535835, 2500000], [537304, 2500000], [536304, 2500000], [536536, 2500000], [536084, 2500000], [536081, 2500000], [536642, 2500000], [536715, 2500000], [536079, 2500000], [536413, 2500000], [536498, 2500000], [536903, 2500000], [535432, 2500000], [537168, 2500000], [535201, 2500000], [536578, 2500000], [535415, 2500000], [537523, 2500000], [536337, 2500000], [536655, 2500000], [536291, 2500000], [535741, 2500000], [536016, 2500000], [536996, 2500000], [535578, 2500000], [536734, 2500000], [536261, 2500000], [537007, 2500000], [536712, 2500000], [536802, 2500000], [537559, 2500000], [536766, 2500000], [537399, 2500000], [535800, 2500000], [536024, 2500000], [536128, 2500000], [537325, 2500000], [535367, 2500000], [536030, 2500000], [536979, 25

[[537557, 2500000], [536540, 2500000], [536315, 2500000], [535980, 2500000], [536430, 2500000], [536081, 2500000], [536814, 2500000], [536146, 2500000], [536483, 2500000], [536304, 2500000], [536715, 2500000], [536079, 2500000], [537304, 2500000], [536749, 2500000], [536903, 2500000], [536642, 2500000], [536030, 2500000], [535432, 2500000], [536498, 2500000], [536375, 2500000], [536084, 2500000], [537326, 2500000], [535578, 2500000], [535637, 2500000], [536578, 2500000], [535415, 2500000], [537168, 2500000], [536979, 2500000], [536291, 2500000], [535741, 2500000], [536146, 2500000], [536580, 2500000], [535835, 2500000], [536413, 2500000], [537523, 2500000], [535201, 2500000], [537007, 2500000], [536911, 2500000], [536784, 2500000], [536261, 2500000], [537559, 2500000], [536766, 2500000], [536536, 2500000], [536016, 2500000], [537399, 2500000], [537428, 2500000], [536712, 2500000], [536405, 2500000], [535664, 2500000], [536128, 2500000], [537325, 2500000], [536024, 2500000], [536396, 25

## Map

In [9]:
%%timeit
x = [e[0] for e in l]
y = [e[1] for e in l] 
result_list = map(monte_carlo, x, y)

hits = 0
trys = 0

for result_tuple in result_list:
    tuple_hits, tuple_trys = result_tuple
    hits += tuple_hits
    trys += tuple_trys
    
print((trys-hits)/trys * 4)

#23.9 s ± 205 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#23.5 s ± 86.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
23.5 s ± 86.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
process_pool = multiprocessing.Pool(processes = nrOfCores)
result_list = process_pool.starmap(monte_carlo, l)
process_pool.close()

hits = 0
trys = 0

for result_tuple in result_list:
    tuple_hits, tuple_trys = result_tuple
    hits += tuple_hits
    trys += tuple_trys
    
print((trys-hits)/trys * 4)

#5.88 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.81 s ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#5.74 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
3.1415428
5.74 s ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
